In [ ]:
#import dependancies
from census import Census
import us
import pandas as pd

In [ ]:
#Starting variables
apikey = ""
fields = ['NAME', 'B01001_007E', 'B01001_008E']#fields used for census data

In [ ]:
#function to create df of state populations
#takes a list of population data and the year
#returns a data fram with state populations and the year of the population
def create_year_pop_df(ls,year):
    #create the df
    df = pd.DataFrame(ls)
    #create population column
    df['Population Over 65'] = df['B01001_007E']+df['B01001_008E']
    #create year column
    df['Year'] = year
    #drop unneeded "states"
    df.drop(df.index[df['NAME'] == 'District of Columbia'], inplace=True)
    df.drop(df.index[df['NAME'] == 'Puerto Rico'], inplace=True)
    #order columns and drop unneeded ones
    dfother = df[['NAME','state','Population Over 65', 'Year']]
    return dfother

In [ ]:
#funtion tocombine all of the year data frames into one dataframe
#takes a list of population information
#returns a combined dataframe
def create_dfs(poy):
    #set years of state data
    years = [y for y in range(2014,2024)]
    #create starting df
    df1 = create_year_pop_df(poy[0],2014)
    #loop through the years of dataframes
    for x in range(1,len(years)):
        #create dataframe
        df2 = create_year_pop_df(poy[x],years[x])
        #concatinate dataframes
        df1 = pd.concat([df1,df2])
    return(df1)

In [ ]:
#create census instance
c = Census(apikey)
#create mapping of fips and abbreviation
abfips = us.states.mapping('fips','abbr')
popsoveryear = []#placeholder for data
tracker = 0#keep track of iterations
#loop through years for data
for y in range(2014,2024):
    #pull data from census
    popsoveryear.append(c.acs5.get((fields),
          {'for': 'state:*'},year = y))
    #increment tracker
    tracker+=1
    #create the dataframe
df8 = create_dfs(popsoveryear)
#save dataframe
df8.to_csv('./CSVs/StatePopulations.csv',header=True,index=False)